In [1]:
import pandas as pd
import numpy as np
from functools import reduce

In [2]:
# files
main_path = r"/path_to_files"
test_file = main_path + "/comparison/240209_full/test_metrics.csv"
dip_file = main_path + "/comparison/240212_diploid/diploid_pairwiseMetrics_240212.csv"
full_file = main_path + "/comparison/240209_full/full_pairwiseMetrics_240209.csv"

In [3]:
metrics_t = pd.read_csv(test_file)
metrics_t.replace('na', np.nan, inplace=True)
metrics_t.head()

,Unnamed: 0,gene,program1,program2,RS,ARS,intersection,union,Jaccard,p1_num_AT,p1_AT_genes,p2_num_AT,p2_AT_genes
0,0,AT1G31370.1,BR,OFb,0.12536723465626837,0.0012752262772070281,12,194,0.061855670103092786,26,"AT3G58420.1,AT3G58270.3,AT3G58380.1,AT3G58260....",4,AT1G31390.1;AT1G31400.1;AT1G31380.1;AT1G31370.1
1,1,AT1G31370.1,BR,OFd,0.11292131830564607,0.0006154338653651564,15,194,0.07731958762886598,26,"AT3G58420.1,AT3G58270.3,AT3G58380.1,AT3G58260....",4,AT1G31390.1;AT1G31400.1;AT1G31380.1;AT1G31370.1
2,2,AT1G31370.1,BR,OFm,0.08685433470434271,-0.00021956134779301637,17,194,0.08762886597938144,26,"AT3G58420.1,AT3G58270.3,AT3G58380.1,AT3G58260....",4,AT1G31390.1;AT1G31400.1;AT1G31380.1;AT1G31370.1
3,3,AT1G31370.1,BR,SPd,NaN,NaN,0,193,0.0,26,"AT3G58420.1,AT3G58270.3,AT3G58380.1,AT3G58260....",0,NaN
4,4,AT1G31370.1,BR,SPm,NaN,NaN,0,193,0.0,26,"AT3G58420.1,AT3G58270.3,AT3G58380.1,AT3G58260....",0,NaN


## Remove duplicate entries from comparisons with BR outputs

In [ ]:
# function 1: to remove the duplicate entries
# for BR - find the one that has the highest Jaccard value - likely the correct comparison with BR
# reason for this - BR will assign a gene to multiple clusters depending on its segments

def removeDuplicateEntries(subset):
    uniqueEntries = {}
    uniqueGene = []
    duplicateEntries = {}
    
    subset[["Jaccard"]] = subset[["Jaccard"]].apply(pd.to_numeric, errors='ignore')
    
    for index, row in subset.iterrows():
        #key1 = (row[2], row[3])
        ATgene = row[1]
        
        if ATgene not in uniqueGene:
            uniqueGene.append(ATgene)

            geneSubset = subset[(subset["gene"] == ATgene)]

            for index, row in geneSubset.iterrows():
                key1 = (row[2], row[3])
                keySubset = geneSubset[(geneSubset["program1"] == row[2]) & (geneSubset["program2"] == row[3])]
                
                if keySubset.shape[0] == 1:
                    if key1 not in uniqueEntries:
                        uniqueEntries[key1] = [keySubset]
                    else:
                        uniqueEntries[key1].append(keySubset)

                else:
                    #print(keySubset)
                    #keySubset[["Jaccard"]] = keySubset[["Jaccard"]].apply(pd.to_numeric, errors='ignore')
                    maxJaccard = keySubset.nlargest(1,['Jaccard'])
                    #print(maxJaccard)
                    if key1 not in uniqueEntries:
                        uniqueEntries[key1] = [maxJaccard]
                    else:
                        uniqueEntries[key1].append(maxJaccard)
                    
                    if key1 not in duplicateEntries:
                        duplicateEntries[key1] = [ATgene]
                    else:
                        duplicateEntries[key1].append(ATgene)
                        
    return(uniqueEntries, uniqueGene, duplicateEntries)
    

In [ ]:
dictEntries_d, uniqueList_d, duplicateDict_d = removeDuplicateEntries(subset_d)

In [ ]:
# double check which ones have duplicate entries
for pairKey, metricRow in duplicateDict_d.items():
    print(pairKey)
    print(metricRow)

In [ ]:
# check the size of the new dataframe without duplicate entries
def newMetricsDF(dictEntries):
    listOfDF = []
    for pairKey, metricRow in dictEntries.items():
        print(pairKey)
        columnNames = list(subset_d.columns)
        progPairDF = pd.concat(metricRow)
        progPairDF_nodup = progPairDF.drop_duplicates(subset=progPairDF.columns.difference(['Unnamed: 0']))
        listOfDF.append(progPairDF_nodup)
        print(progPairDF.shape)
        print(progPairDF_nodup.shape)
    
    newMetrics = pd.concat(listOfDF)
    return(newMetrics)

In [ ]:
metrics_dip_nodup = pd.concat(listOfDF)
#metrics_dip_nodup = newMetricsDF(dictEntries_d)
metrics_dip_nodup.shape

In [ ]:
# function: get all values that would be "different"
def getDiffSubset(metrics):
    progPairDF = metrics.copy()
    sub2 = progPairDF[(progPairDF["p1_num_AT"] > 1) | (progPairDF["p2_num_AT"] > 1)]
    print(sub2.shape)
    sub3 = progPairDF[(progPairDF["p1_num_AT"] == 0) & (progPairDF["p2_num_AT"] == 1)]
    #print(sub3)
    print(sub3.shape)
    sub4 = progPairDF[(progPairDF["p1_num_AT"] == 1) & (progPairDF["p2_num_AT"] == 0)]
    #print(sub4)
    print(sub4.shape)
        
    fulldiffAT = pd.concat([sub2, sub3, sub4])
    
    metrics_columns = list(metrics.columns)
    diffDF = pd.DataFrame(diffList, columns = metrics_columns)
    
    return(diffDF)

In [ ]:
summary_dip = getSummaryNumbers(metrics_dip_nodup)

# To correct for metrics (finding averages, then later adding to a new overall metrics table)

In [ ]:
# function 2: creating two dictionaries of dictionaries 
# dictionary of dictionaries
# dictionary 1 = program 1 and program 2 as keys, gene and all other info as values
# dictionary 2 = within each program 1 and program 2
# groups of arabidopsis genes (the union of p1 and p2) as keys, all other info as values (in a list)

#output_file = main_path + "/comparison/240212_diploid/dictionarySets_diploid_240221_3.txt"
#file_to_write = open(output_file, "a+")
def createDictionaries(subset):
    programPairsSame = {}
    programPairsDiff = {}
    countSameList = {}
    countDiffList = {}
    for index, row in subset.iterrows():
        key1 = (row[2], row[3])
        #file_to_write.write(str(key1))
        ATgene = row[1]
        p1_num = row[-4]
        p2_num = row[-2]
        p1_AT = row[-3]
        p2_AT = row[-1]
        #file_to_write.write(str(p1_num) + "," + str(p2_num)+"\n")
        if p1_num == 0 and p2_num >= 1:
            ATunion = p2_AT.split(";")
            ATunion.sort()
            
            if key1 not in countDiffList:
                countDiffList[key1] = [ATgene]
            else:
                countDiffList[key1].append(ATgene)
                
            if key1 not in programPairsDiff:
                programPairsDiff[key1] = [set(ATunion)]
                #file_to_write.write("1a: " + ATgene + "\n")
            else:
                count = 0
                for gene in ATunion:
                    for geneSet in programPairsDiff[key1]:
                        if gene in geneSet:
                            count = count + 1
                            ind = programPairsDiff[key1].index(geneSet)
                            
                if count == 0:
                    programPairsDiff[key1].append(set(ATunion))
                else: 
                    programPairsDiff[key1][ind].update(ATunion)
                    #file_to_write.write("1c: "+ ATgene + "\n")
                
        elif p1_num >= 1 and p2_num == 0:
            ATunion = p1_AT.split(",")
            ATunion.sort()
            
            if key1 not in countDiffList:
                countDiffList[key1] = [ATgene]
            else:
                 countDiffList[key1].append(ATgene)
                    
            if key1 not in programPairsDiff:
                programPairsDiff[key1] = [set(ATunion)]
                #file_to_write.write("2a: "+ ATgene + "\n")
            else:
                count = 0
                for gene in ATunion:
                    for geneSet in programPairsDiff[key1]:
                        if gene in geneSet:
                            count = count + 1
                            ind = programPairsDiff[key1].index(geneSet)
                            
                if count == 0:
                    programPairsDiff[key1].append(set(ATunion))
                else: 
                    programPairsDiff[key1][ind].update(ATunion)
                    #file_to_write.write("2c: "+ ATgene + "\n")
                
        elif (p1_num > 1) or (p2_num > 1):
            p1List = p1_AT.split(",") 
            p1List.sort()
            p2List = p2_AT.split(";")
            p2List.sort()
            if p1List == p2List:
                ATsame = list(set(p1List + p2List))
                ATsame.sort()
                
                if key1 not in countSameList: 
                    countSameList[key1] = [ATgene] 
                else:
                    countSameList[key1].append(ATgene)
                
                if key1 not in programPairsSame:
                    programPairsSame[key1] = [set(ATsame)]
                    #file_to_write.write("3a: "+ ATgene + "\n")
                else:
                    programPairsSame[key1].append(set(ATsame))
                    #file_to_write.write("3b: "+ ATgene+ "\n")
                        
            else:
                ATunion = list(set(p1List + p2List))
                ATunion.sort()
                
                if key1 not in countDiffList:
                    countDiffList[key1] = [ATgene]
                else:
                    countDiffList[key1].append(ATgene)
                    
                if key1 not in programPairsDiff:
                    programPairsDiff[key1] = [set(ATunion)]
                    #file_to_write.write("4a: "+ ATgene+ "\n")
                else:
                    count = 0
                    for gene in ATunion:
                        for geneSet in programPairsDiff[key1]:
                            if gene in geneSet:
                                count = count + 1
                                ind = programPairsDiff[key1].index(geneSet)
                    
                    if count == 0:
                        programPairsDiff[key1].append(set(ATunion))
                    else: 
                        programPairsDiff[key1][ind].update(ATunion)
                            #file_to_write.write("4c: "+ ATgene+ "\n")
    
    return(programPairsSame, programPairsDiff, countSameList, countDiffList)

In [ ]:
print("create Dictionary")
sameATnum_d, diffATnum_d, countSame_d, countDiff_d = createDictionaries(metrics_dip_nodup)

In [ ]:
for key, item in countDiff_d.items():
    print(key)
    print(len(item))
    print(len(set(item)))
    uniqueList = []
    dupList = []
    for gene in item:
        if gene not in uniqueList:
            uniqueList.append(gene)
        else: 
            dupList.append(gene)
    print(dupList)

In [ ]:
# def checkNumGenes(dictionary):
    for key, item in dictionary.items():
        print(key)
        print(len(item))
        geneList =[]
        for geneSet in item:
            for gene3 in geneSet:
                geneList.append(gene3)
                
        print(len(geneList))
        print(len(set(geneList)))

In [ ]:
checkNumGenes(diffATnum_d)

In [ ]:
# function 3: checking the genes
def checkGenesIncluded(dictionary1, dictionary2):
    extraGenes = {}
    for key, item in dictionary1.items():
        print(key)
        print(len(item))
        geneList =[]
        for geneSet in item:
            for gene in geneSet:
                geneList.append(gene)
        print(len(geneList))
        print(len(set(geneList)))
    
        genesNotInKey = set()
        for key2, item2 in dictionary2.items():            
            if key2 == key:
                print(key2)
                print(len(item))
                for gene2 in geneList:
                    if gene2 not in item2:
                        genesNotInKey.add(gene2)
        
        print(len(genesNotInKey))
        print(genesNotInKey)
        
        if len(genesNotInKey) > 0:
            if key not in extraGenes:
                extraGenes[key] = genesNotInKey
                
    return(extraGenes)
            

In [ ]:
extraBR_d = checkGenesIncluded(diffATnum_d, countDiff_d)

In [ ]:
print(extraBR_d)

In [ ]:
# function 4: for only the scenario where the AT genes are not exactly the same between the two orthogroups
# iteration/comparison among the "AT groups" to see if there are any duplicate genes between them
# stops only when only one final "AT group" to add to the list

def removeDup_newSet(testList):
    atgenes3 = []
    for at_tuple in testList:
        for tup in at_tuple:
            atgenes3.append(tup)

    #print(len(atgenes3))
    test_copy = testList.copy()

    newTupleList = testList

    while len(atgenes3) != len(set(atgenes3)):
        atgenes = []
        #print(atgenes)
        for i in range(len(newTupleList)):
            for j in range(len(newTupleList)-1):
                atgroup1 = newTupleList[i]
                atgroup2 = newTupleList[j+1]
                if atgroup1 != atgroup2:
                    if len(set(atgroup1).intersection(set(atgroup2))) > 0:
                        #print("at1")
                        #print(atgroup1)
                        #print("at2")
                        #print(atgroup2)
                        #print(atgroup1.intersection(atgroup2))
                        keyunion = list(set(atgroup1).union(set(atgroup2)))
                        keyunion.sort()
                        #print(keyunion)
                        if set(atgroup1) in test_copy:
                            test_copy.remove(set(atgroup1))
                        if set(atgroup2) in test_copy:
                            test_copy.remove(set(atgroup2)) 
                        if set(keyunion) not in test_copy:
                            test_copy.append(set(keyunion))
                        
                        atgenes.append(tuple(keyunion))
                     
                            
        atgenes_nodup = set(atgenes)
        #print(atgenes_nodup)
        
        atgenes3 = []
        for at_tuple in atgenes_nodup:
            for tup in at_tuple:
                atgenes3.append(tup)

        newTupleList = list(atgenes_nodup)

        #print("lengthTuple:")
        print(len(atgenes3))
        print(len(set(atgenes3)))
    
    print(len(testList))
    print(len(test_copy))
    return(test_copy)

# function 5: for only the scenario where the AT genes are not exactly the same between the two orthogroups
# removeDup_newSet function is found in this function
# updating the keys for each program pair

def updateKeys(dictionary):
    newDict = {}

    for programKey, keyList in dictionary.items():
        print(programKey)
        #print(keySet)
        geneList =[]
        for tup in keyList:
            for gene in tup:
                geneList.append(gene)
        print(len(geneList))
        print(len(set(geneList)))
        
        if len(geneList) == len(set(geneList)):
            newDict[programKey] = keyList
        else:
            newKeyList = removeDup_newSet(keyList)
            #print(newKeySet)
            newDict[programKey] = newKeyList
    return(newDict)

In [ ]:
noDupGeneKey = updateKeys(diffATnum_d)

In [ ]:
checkNumGenes(noDupGeneKey)

In [ ]:
# function 6: for both the "same" and "diff" groups
# create a new dictionary with the new keys, especially the updated ones for the "diff" set

def createNewDictionary(dictionary):
    newDictionary = {}
    for programPair, atKeySet in dictionary.items():
        if programPair not in newDictionary:
            newDictionary[programPair] = {}
            for atKey in atKeySet:
                newATkey = list(atKey)
                newATkey.sort()
                newATkey = tuple(newATkey)
                if newATkey not in newDictionary[programPair]:
                    newDictionary[programPair][newATkey] = []
    return(newDictionary)

In [ ]:
# function 7: add values to the dictionary of dictionaries
# values are individual rows
# if the AT gene is found in the AT key, which is the union of AT genes found between the two orthogroups

def addValuesToDictionaries_Diff(subset, dictionary, geneDict):    
    for index, row in subset.iterrows():
        key1 = (row[2], row[3])
        gene = row[1]
        if key1 in geneDict:
            extraGenes = geneDict[key1]
            for ATkey, items in dictionary[key1].items():
                #print(ATkey)
                if gene in ATkey:
                    #print(gene)
                    if gene not in extraGenes:
                        dictionary[key1][ATkey].append(row.tolist())
        else:
            for ATkey, items in dictionary[key1].items():
                #print(ATkey)
                if gene in ATkey:
                    dictionary[key1][ATkey].append(row.tolist())
                
    return(dictionary) 

def addValuesToDictionaries_Same(subset, dictionary):    
    for index, row in subset.iterrows():
        key1 = (row[2], row[3])
        gene = row[1]
        for ATkey, items in dictionary[key1].items():
            #print(ATkey)
            if gene in ATkey:
                #print(gene)
                dictionary[key1][ATkey].append(row.tolist())
    return(dictionary) 

In [ ]:
# function 8: finding the average metric values for each union of AT genes
# taking the average for all of the metrics
# however, hard to find the "average" for the number of AT genes in p1 and p2 and the AT gene compositions
# so taking the values from the first gene in the list

def newAvgList(programPairs):
    programKeys = programPairs.keys()
    newAvgValues = []
    programPairCount = []
    for pairKey in programKeys:
        #print(pairKey)
        countList = []
        for ATkey, items in programPairs[pairKey].items():
            geneOG = ATkey
            #print(ATkey)
            program1 = items[0][2]
            program2 = items[0][3]
            RSavg = sum(float(x[4]) for x in items)/len(items)
            ARSavg = sum(float(x[5]) for x in items)/len(items)
            intersectionAvg = sum(float(x[6]) for x in items)/len(items)
            unionAvg = sum(float(x[7]) for x in items)/len(items)
            JIavg = sum(float(x[8]) for x in items)/len(items)
            p1_num = items[0][9]
            p1_ATlist = items[0][10]
            p2_num = items[0][11]
            p2_ATlist = items[0][12]
            #print([geneOG, program1, program2, RSavg, ARSavg, intersectionAvg, unionAvg,JIavg,p1_num,p1_ATlist,p2_num,p2_ATlist])
            newAvgValues.append([geneOG, program1, program2, RSavg, ARSavg, intersectionAvg, unionAvg,JIavg,p1_num,p1_ATlist,p2_num,p2_ATlist])
            countList.append(len(ATkey))
            
        finalCount = sum(countList) 
        print(finalCount)
        programPairCount.append([pairKey, finalCount])
        
    return(newAvgValues, programPairCount)

In [ ]:
# remainder code for OG with different AT genes
print("create new dictionary of dictionaries with new AT keys")
newProgramPairsDiff_d = createNewDictionary(noDupGeneKey)
print("add values")
diffATvalue = addValuesToDictionaries_Diff(metrics_dip_nodup, newProgramPairsDiff_d, extraBR_d)
print("calc new avg")
diffOGList_d, oriATnum_diff_d = newAvgList(diffATvalue)

In [ ]:
# for OG with same AT numbers/gene composition
print("for same AT numbers")
print('check new gene numbers')
checkNumGenes(sameATnum_d)
print(" create new dicitonary of dictionaries with new AT keys")
newProgramPairsSame_d = createNewDictionary(sameATnum_d)
print("add values")
sameATvalue_d = addValuesToDictionaries_Same(metrics_dip_nodup, newProgramPairsSame_d)
print("calc new avg")
sameOGList_d, oriATnum_same_d = newAvgList(sameATvalue_d)

In [ ]:
#create dataframes for both datasets
columnNames = ["gene","program1","program2","RS","ARS","intersection","union","Jaccard","p1_num_AT",
                   "p1_AT_genes","p2_num_AT","p2_AT_genes"]

sameOGdf_d = pd.DataFrame(sameOGList_d, columns=columnNames)
diffOGdf_d = pd.DataFrame(diffOGList_d, columns=columnNames)


## Summary of number of orthogroups of each category

In [ ]:
# function 1: get the initial summary numbers from the original metrics table
def getSummaryNumbers(metrics):
    programPairs= {}
    for index, row in metrics.iterrows():
        key1 = (row[2], row[3])
        if key1 not in programPairs:
            programPairs[key1] = []
            programPairs[key1].append(row.tolist())
        else:
            programPairs[key1].append(row.tolist())
            
    sumList = []
    for pairKey, metricRow in programPairs.items():
        print(pairKey)
        columnNames = list(metrics.columns)
        progPairDF = pd.DataFrame(metricRow, columns = columnNames)
        print(progPairDF.shape)
        #print(progPairDF)
        sub1 = progPairDF[(progPairDF["p1_num_AT"] == 1) & (progPairDF["p2_num_AT"] == 1)]
        sub1 = sub1.drop_duplicates(subset=sub1.columns.difference(['Unnamed: 0']))
        print(sub1.shape)
        sub2 = progPairDF[(progPairDF["p1_num_AT"] > 1) | (progPairDF["p2_num_AT"] > 1)]
        sub2 = sub2.drop_duplicates(subset=sub2.columns.difference(['Unnamed: 0']))
        print(sub2.shape)
        sub3 = progPairDF[(progPairDF["p1_num_AT"] == 0) & (progPairDF["p2_num_AT"] == 1)]
        sub3 = sub3.drop_duplicates(subset=sub3.columns.difference(['Unnamed: 0']))
        #print(sub3)
        print(sub3.shape)
        sub4 = progPairDF[(progPairDF["p1_num_AT"] == 1) & (progPairDF["p2_num_AT"] == 0)]
        sub4 = sub4.drop_duplicates(subset=sub4.columns.difference(['Unnamed: 0']))
        #print(sub4)
        print(sub4.shape)
        
        fulldiffAT = sub2.shape[0] + sub3.shape[0] + sub4.shape[0]
        
        sumList.append([pairKey, progPairDF.shape[0], sub1.shape[0], fulldiffAT])
    
    summaryDF = pd.DataFrame(sumList, columns = ["programPair","All","oneAT","diffAT"])
    summaryDF["Total"] = summaryDF["oneAT"] + summaryDF["diffAT"]
    summaryDF["PropOne"] = summaryDF["oneAT"]/summaryDF["Total"]
    
    return(summaryDF)

In [ ]:
# function 2: counting the number of orthogroups for comparison from the updated list

def countrowsNewDF(dataframe):
    programPairs= {}
    for index, row in dataframe.iterrows():
        key1 = (row[1], row[2])
        if key1 not in programPairs:
            programPairs[key1] = []
            programPairs[key1].append(row.tolist())
        else:
            programPairs[key1].append(row.tolist())
    
    countList = []
    for pairKey, metricRow in programPairs.items():
        print(pairKey)
        columnNames = list(dataframe.columns)
        progPairDF = pd.DataFrame(metricRow, columns = columnNames)
        progPairDF_nodup = progPairDF.drop_duplicates(subset=progPairDF.columns.difference(['Unnamed: 0']))
        print(progPairDF.shape)
        print(progPairDF_nodup.shape)
        countList.append([pairKey,progPairDF_nodup.shape[0]])
    
    return(countList)

In [ ]:
# function 3: combine all the outputs into a summary dataframe

def combineDF(metricsDF, ori_cts1_s, ori_cts1_d, newSameDF, newDiffDF):
    initialSummary = getSummaryNumbers(metricsDF)
    
    oriCountSame_DF = pd.DataFrame(ori_cts1_s, columns=["programPair","oriATgenes_same"])
    oriCountDiff_DF = pd.DataFrame(ori_cts1_d, columns=["programPair","oriATgenes_diff"])
    
    list_same = countrowsNewDF(newSameDF)
    list_diff = countrowsNewDF(newDiffDF)
    
    newCountSame_DF = pd.DataFrame(list_same, columns=["programPair","combAvg_same"])
    newCountDiff_DF = pd.DataFrame(list_diff, columns=["programPair","combAvg_diff"])
    
    listOfDF = [initialSummary, oriCountSame_DF, newCountSame_DF, oriCountDiff_DF, newCountDiff_DF]
    df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['programPair']), listOfDF)
    
    df_merged["newTotalOG"] = df_merged["oneAT"] + df_merged["combAvg_same"] + df_merged["combAvg_diff"]
    df_merged["oriTotal"] = df_merged["oriATgenes_same"] + df_merged["oriATgenes_diff"] 
    
    return(df_merged)

In [ ]:
# call function to create summary dataframe
diploid_sum = combineDF(metrics_dip_nodup, oriATnum_same_d, oriATnum_diff_d, sameOGdf_d, diffOGdf_d)
summaryOutput_d = main_path + "/comparison/240212_diploid/summary_diploid_240221.csv"
diploid_sum.to_csv(summaryOutput_d, index = False)

# Creating the final updated metrics table for making heatmaps
oneSubset_d = metrics_dip_nodup[(metrics_dip_nodup["p1_num_AT"]== 1) & (metrics_dip_nodup["p2_num_AT"] == 1)]
oneSubset_d = oneSubset_d.drop(["Unnamed: 0"], axis=1)
print(oneSubset_d.shape)
print("concatenate dataframes")
metrics_dfs_d = [oneSubset_d, sameOGdf_d, diffOGdf_d]
metrics_merged_d = pd.concat(metrics_dfs_d)
print(metrics_merged_d.shape)
print("drop duplicates")
nodup_merged_d = metrics_merged_d.drop_duplicates()
print(nodup_merged_d.shape)
newMetricsNoDup_out_d = main_path + "/comparison/240212_diploid/diploid_newMetrics_noDup_240221.csv"
nodup_merged_d.to_csv(newMetricsNoDup_out_d, index=False)

## Generate new metrics dataframe with corrected orthogroup comparisons

In [ ]:
##############
# START HERE #
##############

### DIPLOID set of functions 

In [ ]:
# general code, but first couple of lines () - use the above code/boxes to track or
# make corrections/adjustments when errors popup
main_path = r"/path_to_files"

# read in files
print("read file")
dip_file = main_path + "/comparison/240212_diploid/diploid_pairwiseMetrics_240212.csv"
metrics_dip = pd.read_csv(dip_file)
metrics_dip.replace('na', np.nan, inplace=True)

subset_d = metrics_dip
print("remove duplicate entries of BR genes")
dictEntries_d, uniqueList_d, duplicateDict_d = removeDuplicateEntries(subset_d)
print("concatenate rows for new metrics table")
metrics_dip_nodup = newMetricsDF(dictEntries_d)
print("create Dictionary")
sameATnum_d, diffATnum_d, countSame_d, countDiff_d = createDictionaries(metrics_dip_nodup)


# for OG with different AT numbers/gene composition
print("for diff AT numbers")
print("get dictionary of extra genes for BR to not include later")
extraBR_d = checkGenesIncluded(diffATnum_d, countDiff_d)
print("check number of genes")
checkNumGenes(diffATnum_d)
print("iterative method to get equal genes in dictionary keys of AT genes")
noDupGeneKey = updateKeys(diffATnum_d)
print("check number of genes again")
checkNumGenes(noDupGeneKey)
print("create new dictionary of dictionaries with new AT keys")
newProgramPairsDiff_d = createNewDictionary(noDupGeneKey)
print("add values")
diffATvalue_d = addValuesToDictionaries_Diff(metrics_dip_nodup, newProgramPairsDiff_d, extraBR_d)
print("calc new avg")
diffOGList_d, oriATnum_diff_d = newAvgList(diffATvalue_d)

# for OG with same AT numbers/gene composition
print("for same AT numbers")
print('check new gene numbers')
checkNumGenes(sameATnum_d)
print(" create new dicitonary of dictionaries with new AT keys")
newProgramPairsSame_d = createNewDictionary(sameATnum_d)
print("add values")
sameATvalue_d = addValuesToDictionaries(metrics_dip_nodup, newProgramPairsSame_d)
print("calc new avg")
sameOGList_d, oriATnum_same_d = newAvgList(sameATvalue_d)

create dataframes for both datasets
columnNames = ["gene","program1","program2","RS","ARS","intersection","union","Jaccard","p1_num_AT",
                   "p1_AT_genes","p2_num_AT","p2_AT_genes"]

sameOGdf_d = pd.DataFrame(sameOGList_d, columns=columnNames)
diffOGdf_d = pd.DataFrame(diffOGList_d, columns=columnNames)

# call function to create summary dataframe
diploid_sum = combineDF(metrics_dip_nodup, oriATnum_same_d, oriATnum_diff_d, sameOGdf_d, diffOGdf_d)
summaryOutput_d = main_path + "/comparison/240212_diploid/summary_diploid_240221.csv"
diploid_sum.to_csv(summaryOutput_d, index = False)

# Creating the final updated metrics table for making heatmaps
oneSubset_d = metrics_dip[(metrics_dip["p1_num_AT"]== 1) & (metrics_dip["p2_num_AT"] == 1)]
oneSubset_d = oneSubset_d.drop(["Unnamed: 0"], axis=1)
print(oneSubset_d.shape)
print("concatenate dataframes")
metrics_dfs_d = [oneSubset_d, sameOGdf_d, diffOGdf_d]
metrics_merged_d = pd.concat(metrics_dfs_d)
print(nodup_merged_d.shape)
print("drop duplicates")
nodup_merged_d = metrics_merged_d.drop_duplicates()
print(nodup_merged_d.shape)
newMetricsNoDup_out_d = main_path + "/comparison/240212_diploid/diploid_newMetrics_noDup_240221.csv"
nodup_merged_d.to_csv(newMetricsNoDup_out_d, index=False)

In [ ]:
checkNumGenes(sameATnum_d)

In [ ]:
for key, item in countDiff_d.items():
    print(key)
    print(len(item))
    print(len(set(item)))
    uniqueList = []
    dupList = []
    for gene in item:
        if gene not in uniqueList:
            uniqueList.append(gene)
        else: 
            dupList.append(gene)
    print(dupList)

### for the diploid+higher ploidy "FULL set of functions 

In [ ]:
main_path = r"/path_to_files"

# read in files
print("read file")
full_file = main_path + "/comparison/240209_full/full_pairwiseMetrics_240209.csv"
metrics_full = pd.read_csv(full_file)
metrics_full.replace('na', np.nan, inplace=True)

subset_f = metrics_full
print("remove duplicate entries of BR genes")
dictEntries_f, uniqueList_f, duplicateDict_f = removeDuplicateEntries(subset_f)


In [ ]:
# print("concatenate rows for new metrics table")
metrics_full_nodup = newMetricsDF(dictEntries_f)
print("create Dictionary")
sameATnum_f, diffATnum_f, countSame_f, countDiff_f = createDictionaries(metrics_full_nodup)


In [ ]:
# for OG with different AT numbers/gene composition
print("for diff AT numbers")
print("get dictionary of extra genes for BR to not include later")
extraBR_f = checkGenesIncluded(diffATnum_f, countDiff_f)
print("check number of genes")
checkNumGenes(diffATnum_f)
print("iterative method to get equal genes in dictionary keys of AT genes")
noDupGeneKey_f = updateKeys(diffATnum_f)
print("check number of genes again")
checkNumGenes(noDupGeneKey_f)
print("create new dictionary of dictionaries with new AT keys")
newProgramPairsDiff_f = createNewDictionary(noDupGeneKey_f)
print("add values")
diffATvalue_f = addValuesToDictionaries_Diff(metrics_full_nodup, newProgramPairsDiff_f, extraBR_f)
print("calc new avg")
diffOGList_f, oriATnum_diff_f = newAvgList(diffATvalue_f)

In [ ]:
# for OG with same AT numbers/gene composition
print("for same AT numbers")
print('check new gene numbers')
checkNumGenes(sameATnum_f)
print("create new dicitonary of dictionaries with new AT keys")
newProgramPairsSame_f = createNewDictionary(sameATnum_f)
print("add values")
sameATvalue_f = addValuesToDictionaries_Same(metrics_full_nodup, newProgramPairsSame_f)
print("calc new avg")
sameOGList_f, oriATnum_same_f = newAvgList(sameATvalue_f)


In [ ]:
#create dataframes for both datasets
columnNames = ["gene","program1","program2","RS","ARS","intersection","union","Jaccard","p1_num_AT",
                   "p1_AT_genes","p2_num_AT","p2_AT_genes"]

sameOGdf_f = pd.DataFrame(sameOGList_f, columns=columnNames)
diffOGdf_f = pd.DataFrame(diffOGList_f, columns=columnNames)

# call function to create summary dataframe
full_sum = combineDF(metrics_full_nodup, oriATnum_same_f, oriATnum_diff_f, sameOGdf_f, diffOGdf_f)
summaryOutput_f = main_path + "/comparison/240209_full/summary_full_240221.csv"
full_sum.to_csv(summaryOutput_f, index = False)

# Creating the final updated metrics table for making heatmaps
oneSubset_f = metrics_full_nodup[(metrics_full_nodup["p1_num_AT"]== 1) & (metrics_full_nodup["p2_num_AT"] == 1)]
oneSubset_f = oneSubset_f.drop(["Unnamed: 0"], axis=1)
print(oneSubset_f.shape)
print("concatenate dataframes")
metrics_dfs_f = [oneSubset_f, sameOGdf_f, diffOGdf_f]
metrics_merged_f = pd.concat(metrics_dfs_f)
print(metrics_merged_f.shape)
print("drop duplicates")
nodup_merged_f = metrics_merged_f.drop_duplicates()
print(nodup_merged_f.shape)
newMetricsNoDup_out_f = main_path + "/comparison/240209_full/full_newMetrics_noDup_240221.csv"
nodup_merged_f.to_csv(newMetricsNoDup_out_f, index=False)

In [ ]:
newMetricsNoDup_out_f = main_path + "/comparison/240209_full/full_newMetrics_noDup_240221.csv"
nodup_merged_f.to_csv(newMetricsNoDup_out_f, index=False)

## Entire script put into one

In [ ]:
import pandas as pd
import numpy as np
from functools import reduce

######################
# FUNCTIONS - PART 1 #
######################

# function 1: creating two dictionaries of dictionaries 
# dictionary of dictionaries
# dictionary 1 = program 1 and program 2 as keys, gene and all other info as values
# dictionary 2 = within each program 1 and program 2
# groups of arabidopsis genes (the union of p1 and p2) as keys, all other info as values (in a list)

def createDictionaries(subset):
    programPairsSame = {}
    programPairsDiff = {}
    for index, row in subset.iterrows():
        key1 = (row[2], row[3])
        #print(key1)
        p1_num = row[-4]
        p2_num = row[-2]
        p1_AT = row[-3]
        p2_AT = row[-1]
        #print(p1_num, p2_num)
        if p1_num == 0 and p2_num >= 1:
            ATunion = p2_AT.split(";")
            ATunion.sort()

            if key1 not in programPairsDiff:
                programPairsDiff[key1] = set()
                programPairsDiff[key1].add(tuple(ATunion))
            else:
                programPairsDiff[key1].add(tuple(ATunion))
            
        elif p2_num == 0 and p1_num >= 1:
            ATunion = p1_AT.split(",")
            ATunion.sort()

            if key1 not in programPairsDiff:
                programPairsDiff[key1] = set()
                programPairsDiff[key1].add(tuple(ATunion))
            else:
                programPairsDiff[key1].add(tuple(ATunion))
                
        elif (p1_num > 1) or (p2_num > 1):
            p1List = p1_AT.split(",") 
            p1List.sort()
            p2List = p2_AT.split(";")
            p2List.sort()
            if p1List == p2List:
                ATsame = list(set(p1_AT.split(",") + p2_AT.split(";")))
                ATsame.sort()
                if key1 not in programPairsSame:
                    programPairsSame[key1] = set()
                    programPairsSame[key1].add(tuple(ATsame)) 
                else:
                    programPairsSame[key1].add(tuple(ATsame))
                        
            else:
                ATunion = list(set(p1_AT.split(",") + p2_AT.split(";")))
                ATunion.sort()
                if key1 not in programPairsDiff:
                    programPairsDiff[key1] = set()
                    programPairsDiff[key1].add(tuple(ATunion))
                else:
                    programPairsDiff[key1].add(tuple(ATunion))
                        
    return(programPairsSame, programPairsDiff)

# function 2: for only the scenario where the AT genes are not exactly the same between the two orthogroups
# need to find the most inclusive group of AT genes, for the clusters with different number of AT
# genes in the two orhtogroups

def InclusiveATkey(dictionary):
    
    programKeys = dictionary.keys()
    new_uniqueKeys_dict = {}
    
    for pairKey in programKeys:
        uniqueKeys = {}
        for atkey in dictionary[pairKey]:
            #print(atkey)
            for atgene in atkey:
                if atgene not in uniqueKeys:
                    uniqueKeys[atgene] = []
                    uniqueKeys[atgene].append(atkey)
                else: 
                    uniqueKeys[atgene].append(atkey)

        newKeySet = set()
        for atgene2, listOfATkeys in uniqueKeys.items():
            maxATkey = max(atkey2 for atkey2 in listOfATkeys)
            #print(maxATkey)
            newKeySet.add(maxATkey)
        
        if pairKey not in new_uniqueKeys_dict:
            new_uniqueKeys_dict[pairKey]=set()
        
            for uniqueATkey in newKeySet:
                if uniqueATkey not in new_uniqueKeys_dict[pairKey]:
                    #print(uniqueATkey)
                    new_uniqueKeys_dict[pairKey].add(uniqueATkey)
            
    return(new_uniqueKeys_dict)

# function 3: for only the scenario where the AT genes are not exactly the same between the two orthogroups
# to check number of genes in the keys
def checkNumGenes(dictionary):
    for key, item in dictionary.items():
        print(key)
        print(len(item))
        geneList =[]
        for tup in item:
            for gene in tup:
                geneList.append(gene)
                
        print(len(geneList))
        print(len(set(geneList)))
        
# function 4: for only the scenario where the AT genes are not exactly the same between the two orthogroups
# iteration/comparison among the "AT groups" to see if there are any duplicate genes between them
# stops only when only one final "AT group" to add to the list

def removeDup_newSet(test):
    atgenes3 = []
    for at_tuple in test:
        for tup in at_tuple:
            atgenes3.append(tup)

    #print(len(atgenes3))
    test_copy = test.copy()

    newTupleList = test

    while len(atgenes3) != len(set(atgenes3)):
        atgenes = []
        for atgroup1 in newTupleList:
            for atgroup2 in newTupleList:
                if atgroup1 != atgroup2:
                    if len(set(atgroup1).intersection(set(atgroup2))) > 0:
                        #print(atgroup1)
                        #print(atgroup2)
                        #print(set(x).intersection(set(y)))
                        keyunion = list(set(atgroup1).union(set(atgroup2)))
                        keyunion.sort()
                        #print(keyunion)
                        test_copy.discard(atgroup1)
                        test_copy.discard(atgroup2)
                        atgenes.append(tuple(keyunion))
                        test_copy.add(tuple(keyunion))

        atgenes_nodup = set(atgenes)

        atgenes3 = []
        for at_tuple in atgenes_nodup:
            for tup in at_tuple:
                atgenes3.append(tup)

        newTupleList = atgenes_nodup

        #print("lengthTuple:")
        print(len(atgenes3))
        print(len(set(atgenes3)))
    
    return(test_copy)

# function 5: for only the scenario where the AT genes are not exactly the same between the two orthogroups
# removeDup_newSet function is found in this function
# updating the keys for each program pair

def updateKeys(dictionary):
    newDict = {}

    for programKey, keySet in dictionary.items():
        print(programKey)
        #print(keySet)
        geneList =[]
        for tup in keySet:
            for gene in tup:
                geneList.append(gene)
        print(len(geneList))
        print(len(set(geneList)))
        
        if len(geneList) == len(set(geneList)):
            newDict[programKey] = keySet
        else:
            newKeySet = removeDup_newSet(keySet)
            #print(newKeySet)
            newDict[programKey] = newKeySet
    return(newDict)

# function 6: for both the "same" and "diff" groups
# create a new dictionary with the new keys, especially the updated ones for the "diff" set

def createNewDictionary(dictionary):
    newDictionary = {}
    for programPair, atKeySet in dictionary.items():
        if programPair not in newDictionary:
            newDictionary[programPair] = {}
            for atKey in atKeySet:
                newDictionary[programPair][atKey] = []
    return(newDictionary)

# function 7: add values to the dictionary of dictionaries
# values are individual rows
# if the AT gene is found in the AT key, which is the union of AT genes found between the two orthogroups

def addValuesToDictionaries(subset, dictionary):    
    for index, row in subset.iterrows():
        key1 = (row[2], row[3])
        gene = row[1]
        for ATkey, items in dictionary[key1].items():
            #print(ATkey)
            if gene in ATkey:
                #print(gene)
                #print(row.tolist())
                dictionary[key1][ATkey].append(row.tolist())
    return(dictionary) 

# function 8: finding the average metric values for each union of AT genes
# taking the average for all of the metrics
# however, hard to find the "average" for the number of AT genes in p1 and p2 and the AT gene compositions
# so taking the values from the first gene in the list

def newAvgList(programPairs):
    programKeys = programPairs.keys()
    newAvgValues = []
    programPairCount = []
    for pairKey in programKeys:
        #print(pairKey)
        countList = []
        for ATkey, items in programPairs[pairKey].items():
            geneOG = ATkey
            #print(ATkey)
            program1 = items[0][2]
            program2 = items[0][3]
            RSavg = sum(float(x[4]) for x in items)/len(items)
            ARSavg = sum(float(x[5]) for x in items)/len(items)
            intersectionAvg = sum(float(x[6]) for x in items)/len(items)
            unionAvg = sum(float(x[7]) for x in items)/len(items)
            JIavg = sum(float(x[8]) for x in items)/len(items)
            p1_num = items[0][9]
            p1_ATlist = items[0][10]
            p2_num = items[0][11]
            p2_ATlist = items[0][12]
            #print([geneOG, program1, program2, RSavg, ARSavg, intersectionAvg, unionAvg,JIavg,p1_num,p1_ATlist,p2_num,p2_ATlist])
            newAvgValues.append([geneOG, program1, program2, RSavg, ARSavg, intersectionAvg, unionAvg,JIavg,p1_num,p1_ATlist,p2_num,p2_ATlist])
            countList.append(len(ATkey))
            
        finalCount = sum(countList) 
        print(finalCount)
        programPairCount.append([pairKey, finalCount])
        
    return(newAvgValues, programPairCount)

######################
# FUNCTIONS - PART 2 #
######################

# functions for making summary dataframe

# function 1: get the initial summary numbers from the original metrics table
def getSummaryNumbers(metrics):
    programPairs= {}
    for index, row in metrics.iterrows():
        key1 = (row[2], row[3])
        if key1 not in programPairs:
            programPairs[key1] = []
            programPairs[key1].append(row.tolist())
        else:
            programPairs[key1].append(row.tolist())
            
    sumList = []
    for pairKey, metricRow in programPairs.items():
        print(pairKey)
        columnNames = list(metrics.columns)
        progPairDF = pd.DataFrame(metricRow, columns = columnNames)
        print(progPairDF.shape)
        #print(progPairDF)
        sub1 = progPairDF[(progPairDF["p1_num_AT"] == 1) & (progPairDF["p2_num_AT"] == 1)]
        print(sub1.shape)
        sub2 = progPairDF[(progPairDF["p1_num_AT"] > 1) | (progPairDF["p2_num_AT"] > 1)]
        print(sub2.shape)
        sub3 = progPairDF[(progPairDF["p1_num_AT"] == 0) & (progPairDF["p2_num_AT"] == 1)]
        #print(sub3)
        print(sub3.shape)
        sub4 = progPairDF[(progPairDF["p1_num_AT"] == 1) & (progPairDF["p2_num_AT"] == 0)]
        #print(sub4)
        print(sub4.shape)
        
        fulldiffAT = sub2.shape[0] + sub3.shape[0] + sub4.shape[0]
        
        sumList.append([pairKey, progPairDF.shape[0], sub1.shape[0], fulldiffAT])
    
    summaryDF = pd.DataFrame(sumList, columns = ["programPair","All","oneAT","diffAT"])
    summaryDF["Total"] = summaryDF["oneAT"] + summaryDF["diffAT"]
    summaryDF["PropOne"] = summaryDF["oneAT"]/summaryDF["Total"]
    
    return(summaryDF)

# function 2: counting the number of orthogroups for comparison from the updated list

def countrowsNewDF(dataframe):
    programPairs= {}
    for index, row in dataframe.iterrows():
        key1 = (row[1], row[2])
        if key1 not in programPairs:
            programPairs[key1] = []
            programPairs[key1].append(row.tolist())
        else:
            programPairs[key1].append(row.tolist())
    
    countList = []
    for pairKey, metricRow in programPairs.items():
        print(pairKey)
        columnNames = list(dataframe.columns)
        progPairDF = pd.DataFrame(metricRow, columns = columnNames)
        print(progPairDF.shape)
        countList.append([pairKey,progPairDF.shape[0]])
    
    return(countList)

# function 3: combine all the outputs into a summary dataframe

def combineDF(metricsDF, ori_cts1_s, ori_cts1_d, newSameDF, newDiffDF):
    intialSummary = getSummaryNumbers(metricsDF)
    
    oriCountSame_DF = pd.DataFrame(ori_cts1_s, columns=["programPair","oriATgenes_same"])
    oriCountDiff_DF = pd.DataFrame(ori_cts1_d, columns=["programPair","oriATgenes_diff"])
    
    list_same = countrowsNewDF(newSameDF)
    list_diff = countrowsNewDF(newDiffDF)
    
    newCountSame_DF = pd.DataFrame(list_same, columns=["programPair","combAvg_same"])
    newCountDiff_DF = pd.DataFrame(list_diff, columns=["programPair","combAvg_diff"])
    
    listOfDF = [initialSummary, oriCountSame_DF, newCountSame_DF, oriCountDiff_DF, newCountDiff_DF]
    df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['programPair']), listOfDF)
    
    df_merged["newTotalOG"] = df_merged["oneAT"] + df_merged["combAvg_same"] + df_merged["combAvg_diff"]
    df_merged["oriTotal"] = df_merged["oriATgenes_same"] + df_merged["oriATgenes_diff"] 
    
    return(df_merged)

################
################
## START HERE ##
################
################

# call functions and create outputs

main_path = r"/path_to_files"

##############################
## DIPLOID set of functions ##
##############################

# read in files
print("read file")
dip_file = main_path + "/diploid/output/diploid_pairwiseMetrics_240212.csv"
metrics_dip = pd.read_csv(dip_file)
metrics_dip.replace('na', np.nan, inplace=True)

subset_d = metrics_dip
print("create Dictionary")
sameATnum_d, diffATnum_d = createDictionaries(subset_d)

# for OG with different AT numbers
print("for diff AT numbers")
print("inclusive AT key")
diff_newKeys_d = InclusiveATkey(diffATnum_d)
checkNumGenes(diff_newKeys_d)
print("update keys for diff")
programPairsDiff_update_d = updateKeys(diff_newKeys_d)
print('check new gene numbers')
checkNumGenes(programPairsDiff_update_d)
print("create new dictionary of dictionaries with new AT keys")
newProgramPairsDiff_d = createNewDictionary(programPairsDiff_update_d)
print("add values")
diffATvalue = addValuesToDictionaries(subset_d, newProgramPairsDiff_d)
print("calc new avg")
diffOGList_d, oriATnum_diff_d = newAvgList(diffATvalue_d)

# for OG with same AT numbers/gene composition
print("for same AT numbers")
print("inclusive AT key - shouldn't change")
same_newKeys_d = InclusiveATkey(sameATnum_d)
print(" create new dicitonary of dictionaries with new AT keys")
newProgramPairsSame_d = createNewDictionary(same_newKeys_d)
print("add values")
sameATvalue_d = addValuesToDictionaries(subset_d, newProgramPairsSame_d)
print("calc new avg")
sameOGList_d, oriATnum_same_d = newAvgList(sameATvalue_d)

#create dataframes for both datasets
columnNames = ["gene","program1","program2","RS","ARS","intersection","union","Jaccard","p1_num_AT",
                   "p1_AT_genes","p2_num_AT","p2_AT_genes"]

sameOGdf_d = pd.DataFrame(sameOGList_d, columns=columnNames)
diffOGdf_d = pd.DataFrame(diffOGList_d, columns=columnNames)

sameOGdf_d_nodup = sameOGdf_d.drop_duplicates()
diffOGdf_d_nodup = diffOGdf_d.drop_duplicates()

# call function to create summary dataframe
diploid_sum = combineDF(metrics_dip, oriATnum_same_d, oriATnum_diff_d, sameOGdf_d_nodup, diffOGdf_d_nodup)
summaryOutput_d = main_path + "/diploid/output/summary_diploid_240215.csv"
diploid_sum.to_csv(summaryOutput_d, index = False)

# Creating the final updated metrics table for making heatmaps
oneSubset_d = metrics_dip[(metrics_dip["p1_num_AT"]== 1) & (metrics_dip["p2_num_AT"] == 1)]
oneSubset_d = oneSubset_d.drop(["Unnamed: 0"], axis=1)
print(oneSubset_d.shape)
print("concatenate dataframes")
metrics_dfs_d = [oneSubset_d, sameOGdf_d_nodup, diffOGdf_d_nodup]
metrics_merged_d = pd.concat(metrics_dfs_d)
print(metrics_merged_d.shape)
print("drop duplicates")
nodup_merged_d = metrics_merged_d.drop_duplicates()
print(nodup_merged_d.shape)
newMetricsNoDup_out_d = main_path + "/diploid/output/diploid_newMetrics_noDup_240215.csv"
nodup_merged_d.to_csv(newMetricsNoDup_out_d, index=False)

###########################
## FULL set of functions ##
###########################
# read in files
print("read file")
full_file = main_path + "/fullset/output/full_pairwiseMetrics_240209.csv"
metrics_full = pd.read_csv(full_file)
metrics_full.replace('na', np.nan, inplace=True)

subset_f = metrics_full
print("create Dictionary")
sameATnum_f, diffATnum_f = createDictionaries(subset_f)

# for OG with different AT numbers
print("for diff AT numbers")
print("inclusive AT key")
diff_newKeys_f = InclusiveATkey(diffATnum_f)
checkNumGenes(diff_newKeys_f)
print("update keys for diff")
programPairsDiff_update_f = updateKeys(diff_newKeys_f)
print('check new gene numbers')
checkNumGenes(programPairsDiff_update_f)
print("create new dictionary of dictionaries with new AT keys")
newProgramPairsDiff_f = createNewDictionary(programPairsDiff_update_f)
print("add values")
diffATvalue = addValuesToDictionaries(subset_f, newProgramPairsDiff_f)
print("calc new avg")
diffOGList_f, oriATnum_diff_f = newAvgList(diffATvalue_f)

# for OG with same AT numbers/gene composition
print("for same AT numbers")
print("inclusive AT key - shouldn't change")
same_newKeys_f = InclusiveATkey(sameATnum_f)
print(" create new dicitonary of dictionaries with new AT keys")
newProgramPairsSame_f = createNewDictionary(same_newKeys_f)
print("add values")
sameATvalue_f = addValuesToDictionaries(subset_f, newProgramPairsSame_f)
print("calc new avg")
sameOGList_f, oriATnum_same_f = newAvgList(sameATvalue_f)

#create dataframes for both datasets
columnNames = ["gene","program1","program2","RS","ARS","intersection","union","Jaccard","p1_num_AT",
                   "p1_AT_genes","p2_num_AT","p2_AT_genes"]

sameOGdf_f = pd.DataFrame(sameOGList_f, columns=columnNames)
diffOGdf_f = pd.DataFrame(diffOGList_f, columns=columnNames)

sameOGdf_f_nodup = sameOGdf_f.drop_duplicates()
diffOGdf_f_nodup = diffOGdf_f.drop_duplicates()

# call function to create summary dataframe
full_sum = combineDF(metrics_full, oriATnum_same_f, oriATnum_diff_f, sameOGdf_f_nodup, diffOGdf_f_nodup)
summaryOutput_f = main_path + "/fullset/output/summary_full_240215.csv"
full_sum.to_csv(summaryOutput_f, index = False)

# Creating the final updated metrics table for making heatmaps
oneSubset_f = metrics_full[(metrics_full["p1_num_AT"]== 1) & (metrics_full["p2_num_AT"] == 1)]
oneSubset_f = oneSubset_f.drop(["Unnamed: 0"], axis=1)
print(oneSubset_f.shape)
print("concatenate dataframes")
metrics_dfs_f = [oneSubset_f, sameOGdf_f_nodup, diffOGdf_f_nodup]
metrics_merged_f = pd.concat(metrics_dfs_f)
print(metrics_merged_d.shape)
print("drop duplicates")
nodup_merged_f = metrics_merged_f.drop_duplicates()
print(nodup_merged_f.shape)
newMetricsNoDup_out_f = main_path + "/fullset/output/full_newMetrics_noDup_240215.csv"
nodup_merged_f.to_csv(newMetricsNoDup_out_f, index=False)

